In [1]:
import arcpy
from arcpy import env
import pandas as pd
import numpy as np

In [2]:
env_path = r"F:\ArcgisData\pred_soildeep\tif_file"
env.workspace = env_path
raster_list = arcpy.ListRasters()
print(raster_list)
print(len(raster_list))

['Analytical Hillshading.tif', 'Aspect.tif', 'Channel Network Base Level.tif', 'Channel Network Distance.tif', 'Closed Depressions.tif', 'Convergence Index.tif', 'DEM2.tif', 'LS-Factor.tif', 'MRRTF.tif', 'MRVBF.tif', 'NDVI.tif', 'Plan Curvature.tif', 'PRE.tif', 'Profile Curvature.tif', 'Relative Slope Position.tif', 'Slope.tif', 'TMP.tif', 'Topographic Wetness Index.tif', 'Total Catchment Area.tif', 'Valley Depth.tif']
20


In [3]:
# 检查现有的raster空间范围和像元是否一致

def check_raster_info(raster_path):
    desc = arcpy.Describe(raster_path)
    origin_x = desc.extent.XMin
    origin_y = desc.extent.YMax
    pixel_width = desc.meanCellWidth
    pixel_height = desc.meanCellHeight
    rows = desc.height
    cols = desc.width
    print(origin_x,origin_y,rows,cols,pixel_width,pixel_height)
    return((origin_x,origin_y,rows,cols,pixel_width,pixel_height))

In [4]:
check_list = set()
for one_raster in raster_list:
    check_raster_info(one_raster)
    check_list.add(check_raster_info(one_raster))

204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30.000000000000004 30.0
204276.7025531609 3205642.9676713645 14367 14753 30

In [5]:
check_list

{(204276.7025531609,
  3205642.9676713645,
  14367,
  14753,
  30.000000000000004,
  30.0)}

In [6]:
# 选取标准数据生成lon、lat图层
stander_raster = "DEM2.tif"
stander_desc = arcpy.Describe(stander_raster)
stander_origin_x = stander_desc.extent.XMin
stander_origin_y = stander_desc.extent.YMax
stander_pixel_width = stander_desc.meanCellWidth
stander_pixel_height = stander_desc.meanCellHeight
stander_array = arcpy.RasterToNumPyArray("DEM2.tif")

In [7]:
# 经度
array_x = np.zeros(stander_array.shape,dtype=np.float32)
array_x[:, 0] = stander_origin_x+stander_pixel_width/2
for i in range(1, stander_array.shape[1]):
    array_x[:, i] = array_x[:, i-1] + stander_pixel_width
print(array_x.shape,array_x[0,0],array_x[-1,-1])

(14367, 14753) 204291.7 646851.7


In [8]:
# 纬度
array_y = np.zeros(stander_array.shape,dtype=np.float32)
array_y[0] = stander_origin_y-stander_pixel_height/2
for i in range(1, stander_array.shape[0]):
    array_y[i] = array_y[i-1] - stander_pixel_height
print(array_y.shape,array_y[0][0],array_y[-1][-1])

(14367, 14753) 3205628.0 2774648.0


In [10]:
# 确定范围
env.extent = "DEM2.tif"

In [11]:
original_raster = arcpy.Raster("DEM2.tif")

In [12]:
# 将生成的x,y数组保存为栅格
out_raster_x = arcpy.NumPyArrayToRaster(
    array_x,
    lower_left_corner=original_raster.extent.lowerLeft,
    x_cell_size=original_raster.meanCellWidth,
    y_cell_size=original_raster.meanCellHeight,
    )
out_raster_y = arcpy.NumPyArrayToRaster(
    array_y,
    lower_left_corner=original_raster.extent.lowerLeft,
    x_cell_size=original_raster.meanCellWidth,
    y_cell_size=original_raster.meanCellHeight,
    )

In [13]:
out_raster_x.save("LON.tif")
out_raster_y.save("LAT.tif")

In [15]:
spatial_ref = arcpy.Describe("DEM2.tif").spatialReference

In [16]:
arcpy.DefineProjection_management("LON.tif", spatial_ref)
arcpy.DefineProjection_management("LAT.tif", spatial_ref)

<Result 'F:\\ArcgisData\\pred_soildeep\\tif_file\\LAT.tif'>